In [2]:
#step_1 we need to configure the env and load the dataset

In [3]:
!pip install huggingface-hub

In [4]:
!pip install datasets

  Using cached numpy-2.2.0-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl.metadata (19 kB)
Using cached numpy-2.2.0-cp312-cp312-win_amd64.whl (12.6 MB)
Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl (11.5 MB)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.2 requires catalogue<2.1.0,>=2.0.4, which is not installed.
thinc 8.3.2 requires srsly<3.0.0,>=2.4.0, which is not installed.
wordcloud 1.9.4 requires matplotlib, which is not installed.
thinc 8.3.2 requires numpy<2.1.0,>=2.0.0; python_version >= "3.9", but you have numpy 2.2.0 which is incompatible.


In [5]:
from huggingface_hub import login
print("Hugging Face Hub is working")


C:\Users\c\anaconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Hugging Face Hub is working


In [6]:
!huggingface-cli --help

usage: huggingface-cli <command> [<args>]

positional arguments:
  {download,upload,repo-files,env,login,whoami,logout,auth,repo,lfs-enable-largefiles,lfs-multipart-upload,scan-cache,delete-cache,tag,version,upload-large-folder}
                        huggingface-cli command helpers
    download            Download files from the Hub
    upload              Upload a file or a folder to a repo on the Hub
    repo-files          Manage files in a repo on the Hub
    env                 Print information about the environment.
    login               Log in using a token from
                        huggingface.co/settings/tokens
    whoami              Find out which huggingface.co account you are logged
                        in as.
    logout              Log out
    auth                Other authentication related commands
    repo                {create} Commands to interact with your huggingface.co
                        repos.
    lfs-enable-largefiles
                        Co

In [7]:
!pip install torch
! conda activate

In [8]:
import pandas

In [9]:
from datasets import load_dataset

ds = load_dataset("stanfordnlp/imdb")

In [10]:
import torch



In [11]:
import torch
print(torch.__version__)  # This should print the version of PyTorch
print(torch.cuda.is_available())  # This should print True if CUDA (GPU support) is available, or False if only CPU is available


2.5.1+cpu
False


In [12]:
#step_2 I want to check the structure and context of the dataset
#1)the dataset type is datasetdict that holds multiple datasets.In this case,including train test unsupervised.every dataset include 2 column.one is text
#the other is label .in label column 0 reprensent negative and 1 represent positive.

In [13]:
print(ds)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


In [14]:
print(ds["test"]["text"][0])

I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn't match the background, and painfully one-dimensional characters cannot be overcome with a 'sci-fi' setting. (I'm sure there are those of you out there who think Babylon 5 is good sci-fi TV. It's not. It's clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It's really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it's rubbish as they have to alway

In [15]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

In [16]:
from transformers import pipeline

In [17]:
tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")

In [18]:
import warnings
warnings.filterwarnings("ignore", message=".*weights.*not used.*")


In [19]:
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [20]:
import torch

model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [21]:
print(ds.keys())

dict_keys(['train', 'test', 'unsupervised'])


In [22]:
from transformers import pipeline

In [23]:
print(ds['train'].features)

{'text': Value(dtype='string', id=None), 'label': ClassLabel(names=['neg', 'pos'], id=None)}


In [24]:
print(ds['test'].features)

{'text': Value(dtype='string', id=None), 'label': ClassLabel(names=['neg', 'pos'], id=None)}


In [25]:
print(ds['unsupervised'].features)

{'text': Value(dtype='string', id=None), 'label': ClassLabel(names=['neg', 'pos'], id=None)}


In [26]:
print(ds)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


In [27]:
print(ds['test']['text'][:5])

['I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn\'t match the background, and painfully one-dimensional characters cannot be overcome with a \'sci-fi\' setting. (I\'m sure there are those of you out there who think Babylon 5 is good sci-fi TV. It\'s not. It\'s clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It\'s really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it\'s rubbish as they hav

In [28]:
#step_3 using pre_trained module for NER

In [29]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")
def extract_names(text):
    ner_results = nlp(text)
    names = [result['word'] for result in ner_results if "PER" in result['entity']]
    return " ".join(names)  # Combine subwords into full names

nlp = pipeline("ner", model=model, tokenizer=tokenizer)
example = ds["unsupervised"]["text"][:500]

ner_results = nlp(example)
print(ner_results)


Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


[[{'entity': 'B-PER', 'score': np.float32(0.72771496), 'index': 32, 'word': 'Leon', 'start': 146, 'end': 150}, {'entity': 'B-PER', 'score': np.float32(0.99945563), 'index': 38, 'word': 'Jean', 'start': 174, 'end': 178}, {'entity': 'I-PER', 'score': np.float32(0.9995165), 'index': 39, 'word': 'Reno', 'start': 179, 'end': 183}, {'entity': 'B-PER', 'score': np.float32(0.9991185), 'index': 41, 'word': 'Natalie', 'start': 188, 'end': 195}, {'entity': 'I-PER', 'score': np.float32(0.9989374), 'index': 42, 'word': 'Port', 'start': 196, 'end': 200}, {'entity': 'I-PER', 'score': np.float32(0.4966278), 'index': 43, 'word': '##man', 'start': 200, 'end': 203}, {'entity': 'B-PER', 'score': np.float32(0.9995067), 'index': 62, 'word': 'Anne', 'start': 266, 'end': 270}, {'entity': 'I-PER', 'score': np.float32(0.99938846), 'index': 63, 'word': 'Pa', 'start': 271, 'end': 273}, {'entity': 'I-PER', 'score': np.float32(0.998638), 'index': 64, 'word': '##rilla', 'start': 273, 'end': 278}, {'entity': 'I-PER',